In [1]:
#Violations Dataset
'''
Purpose
The code converts a DataFrame (ace_violations) containing bus-related violation data 
into a GeoJSON file with point features, suitable for mapping in GIS tools like Kepler.gl, 
Leaflet, or Mapbox.
'''

'\nPurpose\nThe code converts a DataFrame (ace_violations) containing bus-related violation data \ninto a GeoJSON file with point features, suitable for mapping in GIS tools like Kepler.gl, \nLeaflet, or Mapbox.\n'

In [2]:
#Import Libraries
import pandas as pd
import os

In [3]:
# Path to data folder relative to the notebook
DATA_DIR = os.path.join("..", "data", "raw")
ace_violations = pd.read_csv(os.path.join(DATA_DIR, "ACE_violations.csv"))
#ace_violations.head()

In [4]:
import json
import math

features = []

for _, row in ace_violations.iterrows():
    try:
        v_lat, v_lon = row["Violation Latitude"], row["Violation Longitude"]

        # Skip rows with bad coords
        if (
            pd.isna(v_lat) or pd.isna(v_lon) or
            not (math.isfinite(v_lat) and math.isfinite(v_lon))
        ):
            continue

        # Properties (convert everything to string to avoid JSON issues)
        props = {col: str(row[col]) for col in [
            "Violation ID",
            "Vehicle ID",
            "First Occurrence",
            "Last Occurrence",
            "Violation Status",
            "Violation Type",
            "Bus Route ID",
            "Stop ID",
            "Stop Name"
        ] if col in row}

        feature = {
            "type": "Feature",
            "properties": props,
            "geometry": {
                "type": "Point",
                "coordinates": [float(v_lon), float(v_lat)]
            }
        }
        features.append(feature)
    except Exception as e:
        print(f"Skipping row due to error: {e}")

geojson = {
    "type": "FeatureCollection",
    "features": features
}

with open("/Users/danielbrown/Desktop/datathon_project/data/processed/violations.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, indent=2, ensure_ascii=False)

print(f"✅ Saved {len(features)} point features to violations_points.geojson")



✅ Saved 185059 point features to violations_points.geojson


In [5]:
# Preview first 3 features
preview = {
    "type": "FeatureCollection",
    "features": features[:3]
}

#print(json.dumps(preview, indent=2, ensure_ascii=False))


In [34]:
#Bus Segment Speeds
'''
Purpose
This code processes MTA bus route segment speeds and converts them into a GeoJSON file 
of line segments along each bus route, including aggregated speed and trip information. 
This is useful for mapping bus route performance in GIS tools like Kepler.gl or Mapbox.
'''

'\nPurpose\nThis code processes MTA bus route segment speeds and converts them into a GeoJSON file \nof line segments along each bus route, including aggregated speed and trip information. \nThis is useful for mapping bus route performance in GIS tools like Kepler.gl or Mapbox.\n'

In [35]:
# Path to data folder relative to the notebook
DATA_DIR = os.path.join("..", "data", "raw")

In [36]:
segment_speed_2020_2024 = pd.read_csv(os.path.join(DATA_DIR, "segment_speed_2023_2024.csv"))
segment_speed_2025 = pd.read_csv(os.path.join(DATA_DIR, "segment_speed_2025.csv"))
# Concatenate vertically (stack rows)
segment_speed_all = pd.concat([segment_speed_2020_2024, segment_speed_2025], ignore_index=True)

# Optional: check the combined shape
#print(segment_speed_all.shape)
#print(segment_speed_all.head())
df = segment_speed_all

In [9]:
import pandas as pd
import json
from math import radians, sin, cos, sqrt, atan2

# --- 1. Load GTFS shapes and trips ---
shapes = pd.read_csv("/Users/danielbrown/Desktop/gtfs_bx-3/shapes.txt")
shapes = shapes.sort_values(["shape_id", "shape_pt_sequence"])

trips = pd.read_csv("/Users/danielbrown/Desktop/gtfs_bx-3/trips.txt")

# --- 2. Load speeds ---
speeds = pd.read_csv(
    "/Users/danielbrown/Desktop/MTA_Bus_Route_Segment_Speeds__Beginning_2025_20250920.csv",
    parse_dates=["Timestamp"]
)

# --- 2b. Aggregate speeds by Next Timepoint Stop Name and Direction ---
agg_speeds = (
    speeds.groupby(["Next Timepoint Stop Name", "Direction"], as_index=False)
    .agg(
        avg_speed=("Average Road Speed", "mean"),
        total_trips=("Bus Trip Count", "sum"),
        start_lat=("Timepoint Stop Latitude", "first"),
        start_lon=("Timepoint Stop Longitude", "first"),
        end_lat=("Next Timepoint Stop Latitude", "first"),
        end_lon=("Next Timepoint Stop Longitude", "first"),
        route_id=("Route ID", "first")
    )
)

# --- 2c. Map directions and add segment numbers ---
dir_map = {'W': 0, 'E': 1}  # adjust if needed
agg_speeds['direction_id'] = agg_speeds['Direction'].map(dir_map)

# Number segments within each route + direction
agg_speeds = agg_speeds.sort_values(["route_id", "direction_id"])
agg_speeds["segment_number"] = (
    agg_speeds.groupby(["route_id", "direction_id"]).cumcount() + 1
)

# --- 3. Helper functions ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi, dlambda = radians(lat2 - lat1), radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(phi1) * cos(phi2) * sin(dlambda/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))

def closest_index(lat, lon, coords):
    return min(range(len(coords)), key=lambda i: haversine(lat, lon, coords[i][1], coords[i][0]))

# --- 4. Map route+direction to shape_id ---
direction_map = trips.groupby(['route_id', 'direction_id'])['shape_id'].first().to_dict()

# --- 5. Build GeoJSON ---
features = []

for _, row in agg_speeds.iterrows():
    route_dir_key = (row["route_id"], row["direction_id"])
    if route_dir_key not in direction_map:
        continue
    shape_id = direction_map[route_dir_key]
    route_shape = shapes[shapes["shape_id"] == shape_id]
    shape_coords = list(zip(route_shape["shape_pt_lon"], route_shape["shape_pt_lat"]))

    # Find indices along shape for stop and next stop
    i1 = closest_index(row["start_lat"], row["start_lon"], shape_coords)
    i2 = closest_index(row["end_lat"], row["end_lon"], shape_coords)
    if i1 > i2:
        i1, i2 = i2, i1

    # Fallback if points are the same
    if i1 == i2:
        segment_coords = [(row["start_lon"], row["start_lat"]), (row["end_lon"], row["end_lat"])]
    else:
        segment_coords = shape_coords[i1:i2+1]

    features.append({
        "type": "Feature",
        "geometry": {"type": "LineString", "coordinates": segment_coords},
        "properties": {
            "route_id": row["route_id"],
            "direction": row["Direction"],
            "segment_number": int(row["segment_number"]),
            "speed": row["avg_speed"],
            "trips": row["total_trips"]
        }
    })

geojson = {"type": "FeatureCollection", "features": features}

with open("/Users/danielbrown/Desktop/bx12_speeds_shapes_directions_agg_1.geojson", "w") as f:
    json.dump(geojson, f)

print(f"GeoJSON created with {len(features)} features")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/danielbrown/Desktop/MTA_Bus_Route_Segment_Speeds__Beginning_2025_20250920.csv'

In [ ]:
#Visualizing communities around bus routes
'''
Purpose
This script processes median household income data for NYC neighborhoods and merges 
it with CDTA (Community District Tabulation Areas) geometries, preparing a clean dataset 
suitable for mapping and spatial analysis.
'''

In [ ]:
import pandas as pd

# File paths
median_income_fp = '/Users/danielbrown/Desktop/Median_Incomes - Sheet1.csv'
cdta_fp = '/Users/danielbrown/Desktop/2020_Community_District_Tabulation_Areas__CDTAs__20250920.csv'

# Load CSVs
median_income_df = pd.read_csv(median_income_fp)
cdta_df = pd.read_csv(cdta_fp)

# Quick check
# Check columns
print("Median Income Columns:\n", median_income_df.columns, "\n")
print("CDTA Columns:\n", cdta_df.columns, "\n")

# Get the size of the dataframe
rows, cols = cdta_df.shape
print(f"Merged DataFrame size: {rows} rows x {cols} columns")


In [ ]:
# Step 1: Extract the code in parentheses
median_income_df['Code'] = median_income_df['Location'].str.extract(r'\((.*?)\)')

# Step 2: Replace letters only (prefix replacements)
letter_map = {
    "K": "BK",
    "S": "SI",
    "Q": "QN",
    "B": "BX",
    "M": "MN"
}

# Function to replace letters while keeping numbers
def replace_prefix(code):
    if pd.isna(code):
        return code
    for old, new in letter_map.items():
        if code.startswith(old):
            return new + code[len(old):]
    return code  # leave as-is if no match

median_income_df['Code'] = median_income_df['Code'].apply(replace_prefix)

# Optional: check first few rows
print(median_income_df.head())

# Get the size of the dataframe
rows, cols = median_income_df.shape
print(f"Merged DataFrame size: {rows} rows x {cols} columns")

# Step 3: Export to CSV
#output_fp = '/Users/danielbrown/Desktop/median_income_with_codes.csv'
#median_income_df.to_csv(output_fp, index=False)
#print(f"Saved updated CSV to {output_fp}")
#print("Median Income Columns:\n", median_income_df.columns, "\n")

In [ ]:
# Inner join on Code -> CDTA2020
merged_df = median_income_df.merge(
    cdta_df,
    left_on='Code',
    right_on='CDTA2020',
    how='left'
)

# Sort the DataFrame by 'Code'
merged_df = merged_df.sort_values(by='Code').reset_index(drop=True)

merged_df = merged_df[['Location','All Households','Families','Families with Children','Families without Children','Code','the_geom']]

# Step 1: Strip $ and commas from income columns and convert to numeric
income_cols = ['All Households', 'Families', 'Families with Children', 'Families without Children']
for col in income_cols:
    merged_df[col] = merged_df[col].replace('[\$,]', '', regex=True).astype(float)

# Step 2: Ensure string columns
string_cols = ['Location', 'Code', 'the_geom']
for col in string_cols:
    merged_df[col] = merged_df[col].astype(str)

# Quick check
print(merged_df.head())
print(merged_df.columns)
# Get the size of the dataframe
rows, cols = merged_df.shape
print(f"Merged DataFrame size: {rows} rows x {cols} columns")

# Optional: export to CSV
output_fp = '/Users/danielbrown/Desktop/median_income_cdta_merged.csv'
merged_df.to_csv(output_fp, index=False)
print(f"Saved merged CSV to {output_fp}")

In [ ]:
#Census Tract Data and Commute Times

In [37]:
import pandas as pd
import os

# Path to data folder relative to the notebook
DATA_DIR = os.path.join("..", "data", "raw")
travel_time_df = pd.read_csv(os.path.join(DATA_DIR, "NTAD_Travel_Time _Work.csv"))
census_tracts_df = pd.read_csv(os.path.join(DATA_DIR, "2020_Census_Tracts_20250921.csv"))


# Quick check of column names
#print("Travel Time Columns:", travel_time_df.columns.tolist())
#print("Census Tracts Columns:", census_tracts_df.columns.tolist())

# Show quick preview + shape
#print(travel_time_df.head())
#print(f"Merged DataFrame shape: {travel_time_df.shape}")

#print(census_tracts_df.head())
print(f"Merged DataFrame shape: {census_tracts_df.shape}")

Merged DataFrame shape: (2325, 14)


In [38]:
travel_time_df['% of workers with 45+ minutes commute'] = travel_time_df['% of workers with commute of 45 to 59 minutes'] + travel_time_df['% of workers with commute of 60 to 89 minutes'] + travel_time_df['% of workers with commute of 90 or more minutes']




In [39]:
# Left join: keep all census tracts (CTLabel), bring in matching travel times (TRACTID)
merged_df = census_tracts_df.merge(
    travel_time_df,
    how="left",
    left_on="CTLabel",
    right_on="TRACTID"
)

# Reset index for cleanliness
merged_df = merged_df.reset_index(drop=True)

merged_df = merged_df[['NTAName','CTLabel','% of workers with 45+ minutes commute','the_geom']]

# Show quick preview + shape
print(merged_df.head())
print(f"Merged DataFrame shape: {merged_df.shape}")


# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Construct the full path to the output file
output_path = os.path.join(project_root, "data", "processed", "census_merged_df.csv")

# Save the CSV
merged_df.to_csv(output_path, index=False)


                                             NTAName  CTLabel  \
0  The Battery-Governors Island-Ellis Island-Libe...     1.00   
1  The Battery-Governors Island-Ellis Island-Libe...     1.00   
2  The Battery-Governors Island-Ellis Island-Libe...     1.00   
3  The Battery-Governors Island-Ellis Island-Libe...     1.00   
4                                    Lower East Side    14.01   

   % of workers with 45+ minutes commute  \
0                                  16.36   
1                                  13.59   
2                                    NaN   
3                                    NaN   
4                                  33.86   

                                            the_geom  
0  MULTIPOLYGON (((-74.04387761639944 40.69018767...  
1  MULTIPOLYGON (((-74.04387761639944 40.69018767...  
2  MULTIPOLYGON (((-74.04387761639944 40.69018767...  
3  MULTIPOLYGON (((-74.04387761639944 40.69018767...  
4  MULTIPOLYGON (((-73.9883662631772 40.716445702...  
Merged DataFra